In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import os
from absl import flags
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import estimator as tf_estimator

# from projects.fashionpedia.configs import factory
# from projects.fashionpedia.serving import serving
from configs import factory
from serving import serving
from hyperparameters import params_dict
from tensorflow.python.ops import control_flow_util  # pylint: disable=g-direct-tensorflow-import

def export(export_dir,
           checkpoint_path,
           model,
           config_file='',
           params_override='',
           use_tpu=False,
           batch_size=1,
           image_size=(640, 640),
           input_type='image_tensor',
           input_name='input',
           output_image_info=True,
           output_normalized_coordinates=False,
           cast_num_detections_to_float=False,
           cast_detection_classes_to_float=False):
  """Exports the SavedModel."""
  control_flow_util.enable_control_flow_v2()

  params = factory.config_generator(model)
  if config_file:
    params = params_dict.override_params_dict(
        params, config_file, is_strict=True)
  # Use `is_strict=False` to load params_override with run_time variables like
  # `train.num_shards`.
  params = params_dict.override_params_dict(
      params, params_override, is_strict=False)
  if not use_tpu:
    params.override({
        'architecture': {
            'use_bfloat16': use_tpu,
        },
    }, is_strict=True)
  if batch_size is None:
    params.override({
        'postprocess': {
            'use_batched_nms': True,
        }
    })
  params.validate()
  params.lock()

  model_params = dict(
      params.as_dict(),
      use_tpu=use_tpu,
      mode=tf_estimator.ModeKeys.PREDICT,
      transpose_input=False)
  tf.logging.info('model_params is:\n %s', model_params)

  if model in ['attribute_mask_rcnn']:
    model_fn = serving.serving_model_fn_builder(
        use_tpu, output_image_info, output_normalized_coordinates,
        cast_num_detections_to_float, cast_detection_classes_to_float)
    serving_input_receiver_fn = functools.partial(
        serving.serving_input_fn,
        batch_size=batch_size,
        desired_image_size=image_size,
        stride=(2 ** params.architecture.max_level),
        input_type=input_type,
        input_name=input_name)
  else:
    raise ValueError('The model type `{} is not supported.'.format(model))

  print(' - Setting up TPUEstimator...')
  estimator = tf_estimator.tpu.TPUEstimator(
      model_fn=model_fn,
      model_dir=None,
      config=tf_estimator.tpu.RunConfig(
          tpu_config=tf_estimator.tpu.TPUConfig(iterations_per_loop=1),
          master='local',
          evaluation_master='local'),
      params=model_params,
      use_tpu=use_tpu,
      train_batch_size=batch_size,
      predict_batch_size=batch_size,
      export_to_tpu=use_tpu,
      export_to_cpu=True)

  print(' - Exporting the model...')

  dir_name = os.path.dirname(export_dir)
  print(dir_name)

  if not tf.gfile.Exists(dir_name):
    print('creating folder')
    tf.logging.info('Creating base dir: %s', dir_name)
    tf.gfile.MakeDirs(dir_name)

  export_path = estimator.export_saved_model(
      export_dir_base=dir_name,
      serving_input_receiver_fn=serving_input_receiver_fn,
      checkpoint_path=checkpoint_path)

  tf.logging.info(
      'Exported SavedModel to %s, renaming to %s',
      export_path, export_dir)

  if tf.gfile.Exists(export_dir):
    tf.logging.info('Deleting existing SavedModel dir: %s', export_dir)
    tf.gfile.DeleteRecursively(export_dir)

  tf.gfile.Rename(export_path, export_dir)
    
# export("saved_models",
#      "checkpoints/fashionpedia-spinenet-143/model.ckpt",
#      "attribute_mask_rcnn",
#      "configs/yaml/spinenet143_amrcnn.yaml",)

export("saved_models2",
     "tf2model/tf2_model-1.data-00000-of-00001",
     "attribute_mask_rcnn",
     "configs/yaml/spinenet143_amrcnn.yaml",)

C:\Users\Kenji\miniforge3\envs\project\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


INFO:tensorflow:model_params is:
 {'type': 'attribute_mask_rcnn', 'model_dir': '', 'use_tpu': False, 'isolate_session_state': False, 'architecture': {'parser': 'attribute_maskrcnn_parser', 'backbone': 'spinenet', 'min_level': 3, 'max_level': 7, 'use_bfloat16': False, 'space_to_depth_block_size': 1, 'multilevel_features': 'identity', 'include_mask': True, 'mask_target_size': 28, 'num_classes': 47, 'num_attributes': 294}, 'attribute_maskrcnn_parser': {'output_size': [1280, 1280], 'rpn_match_threshold': 0.7, 'rpn_unmatched_threshold': 0.3, 'rpn_batch_size_per_im': 256, 'rpn_fg_fraction': 0.5, 'aug_rand_hflip': True, 'aug_scale_min': 0.5, 'aug_scale_max': 2.0, 'skip_crowd_during_training': True, 'max_num_instances': 100, 'mask_crop_size': 112}, 'anchor': {'num_scales': 1, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_size': 3.0}, 'fpn': {'fpn_feat_dims': 256, 'use_separable_conv': False, 'use_batch_norm': True}, 'nasfpn': {'fpn_feat_dims': 256, 'num_repeats': 5, 'use_separable_conv': False, 'i

INFO:tensorflow:model_params is:
 {'type': 'attribute_mask_rcnn', 'model_dir': '', 'use_tpu': False, 'isolate_session_state': False, 'architecture': {'parser': 'attribute_maskrcnn_parser', 'backbone': 'spinenet', 'min_level': 3, 'max_level': 7, 'use_bfloat16': False, 'space_to_depth_block_size': 1, 'multilevel_features': 'identity', 'include_mask': True, 'mask_target_size': 28, 'num_classes': 47, 'num_attributes': 294}, 'attribute_maskrcnn_parser': {'output_size': [1280, 1280], 'rpn_match_threshold': 0.7, 'rpn_unmatched_threshold': 0.3, 'rpn_batch_size_per_im': 256, 'rpn_fg_fraction': 0.5, 'aug_rand_hflip': True, 'aug_scale_min': 0.5, 'aug_scale_max': 2.0, 'skip_crowd_during_training': True, 'max_num_instances': 100, 'mask_crop_size': 112}, 'anchor': {'num_scales': 1, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_size': 3.0}, 'fpn': {'fpn_feat_dims': 256, 'use_separable_conv': False, 'use_batch_norm': True}, 'nasfpn': {'fpn_feat_dims': 256, 'num_repeats': 5, 'use_separable_conv': False, 'i

 - Setting up TPUEstimator...



INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Kenji\\AppData\\Local\\Temp\\tmprruk_o4p', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'local', '_evaluation_master': 'local', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1, num_shards=None, num_core

...

INFO:tensorflow:Signatures INCLUDED in export for Eval: None


INFO:tensorflow:Signatures INCLUDED in export for Eval: None


INFO:tensorflow:Restoring parameters from tf2model/tf2_model-1.data-00000-of-00001


INFO:tensorflow:Restoring parameters from tf2model/tf2_model-1.data-00000-of-00001


DataLossError: 2 root error(s) found.
  (0) Data loss: Unable to open table file tf2model\tf2_model-1.data-00000-of-00001: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
	 [[node save/RestoreV2 (defined at \miniforge3\envs\project\lib\site-packages\tensorflow_estimator\python\estimator\estimator.py:970) ]]
  (1) Data loss: Unable to open table file tf2model\tf2_model-1.data-00000-of-00001: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
	 [[node save/RestoreV2 (defined at \miniforge3\envs\project\lib\site-packages\tensorflow_estimator\python\estimator\estimator.py:970) ]]
	 [[save/RestoreV2/_13]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'save/RestoreV2':
  File "\miniforge3\envs\project\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, 